In [2]:
%%bash
mkdir -p sample
echo "" > sample/__init__.py

In [9]:
# %%writefile train/model_definition.py
# import tensorflow as tf
# import numpy as np

# #Get data

# (x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

# # add empty color dimension
# x_train = np.expand_dims(x_train, -1)
# x_test = np.expand_dims(x_test, -1)

# def create_model():
#     model = tf.keras.models.Sequential()
#     model.add(tf.keras.layers.BatchNormalization(input_shape=x_train.shape[1:]))
#     model.add(tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='elu'))
#     model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
#     model.add(tf.keras.layers.Dropout(0.25))

#     model.add(tf.keras.layers.BatchNormalization(input_shape=x_train.shape[1:]))
#     model.add(tf.keras.layers.Conv2D(128, (5, 5), padding='same', activation='elu'))
#     model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
#     model.add(tf.keras.layers.Dropout(0.25))

#     model.add(tf.keras.layers.BatchNormalization(input_shape=x_train.shape[1:]))
#     model.add(tf.keras.layers.Conv2D(256, (5, 5), padding='same', activation='elu'))
#     model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
#     model.add(tf.keras.layers.Dropout(0.25))

#     model.add(tf.keras.layers.Flatten())
#     model.add(tf.keras.layers.Dense(256))
#     model.add(tf.keras.layers.Activation('elu'))
#     model.add(tf.keras.layers.Dropout(0.5))
#     model.add(tf.keras.layers.Dense(10))
#     model.add(tf.keras.layers.Activation('softmax'))
#     return model

Overwriting train/model_definition.py


In [4]:
#%%writefile sample/model.py

import tensorflow as tf
from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers


def main():
    mirrored_strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()
                        
    with mirrored_strategy.scope():
        seq_data = keras.Input(shape=(None, 1),  name="seq_data")
        seq_lengths = keras.Input(shape=(), name="seq_lengths", dtype=tf.int32)

        mask = keras.layers.Lambda(lambda x: tf.sequence_mask(x))(seq_lengths)
        conv = layers.Conv1D(32, 3, strides=1, padding='same', activation='relu')(seq_data)
        rnn = layers.Bidirectional(layers.GRU(32, return_sequences=True))(conv,mask=mask)
        #rnn = layers.Bidirectional(layers.GRU(32, return_sequences=True))(conv)
        dense = layers.Dense(5, name="signal_mask")(rnn)
        model = keras.Model(inputs=[seq_data, seq_lengths], outputs=[dense])

        model.compile(optimizer=tf.keras.optimizers.Adam(
            learning_rate=0.001),
            loss=tf.keras.backend.sparse_categorical_crossentropy
        )

    batch_size = 1
    time_step = 1000
    signal_length = 1000
    sequences = np.random.rand(batch_size, time_step, 1)
    seq_lengths = np.array([signal_length])
    signal_mask = np.random.randint(5, size=(1, time_step))

    dataset = tf.data.Dataset.from_tensor_slices(
        (sequences, seq_lengths,  signal_mask)).repeat(1000).batch(1)\
        .map(lambda a,b,c: ({"seq_data":a,"seq_lengths":b}, {"signal_mask":c}))

    model.fit(dataset, epochs=1)
    
main()

INFO:tensorflow:Using MirroredStrategy with devices ('/device:GPU:0', '/device:GPU:1')
INFO:tensorflow:Single-worker MultiWorkerMirroredStrategy with local_devices = ('/device:GPU:0', '/device:GPU:1'), communication = CollectiveCommunication.AUTO
Cause: could not parse the source code:

        .map(lambda a,b,c: ({"seq_data":a,"seq_lengths":b}, {"signal_mask":c}))

This error may be avoided by creating the lambda in a standalone statement.

To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: could not parse the source code:

        .map(lambda a,b,c: ({"seq_data":a,"seq_lengths":b}, {"signal_mask":c}))

This error may be avoided by creating the lambda in a standalone statement.

To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Instructions for updating:
Use the iterator's `initializer` property instead.
Instructions for updating:
Simply pass a True/False value to the `training` argument of t

ValueError: Graph disconnected: cannot obtain value for tensor Tensor("seq_lengths:0", shape=(None,), dtype=int32) at layer "lambda". The following previous layers were accessed without issue: ['conv1d']

In [12]:
%%writefile config.yaml
trainingInput:
  scaleTier: CUSTOM
  # Configure a master worker with 1 with K80 GPUs
  masterType: n1-highcpu-16
  masterConfig:
    acceleratorConfig:
      count: 2
      type: NVIDIA_TESLA_K80
  # Configure 1 worker(s), each with 2 K80
  workerCount: 2
  workerType: n1-highcpu-16
  workerConfig:
    acceleratorConfig:
      count: 2
      type: NVIDIA_TESLA_K80

Overwriting config.yaml


In [15]:
%%bash

now=$(date +"%Y%m%d_%H%M%S")
JOB_NAME="dist_learning_test$now"

gcloud ai-platform jobs submit training $JOB_NAME \
  --staging-bucket=gs://accelerator_demo5 \
  --package-path=sample \
  --module-name=sample.model \
  --runtime-version=2.1 \
  --python-version=3.7 \
  --region=us-west1 \
  --config config.yaml
  

jobId: dist_learning_test20200805_163954
state: QUEUED


Job [dist_learning_test20200805_163954] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe dist_learning_test20200805_163954

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs dist_learning_test20200805_163954


In [24]:
import tensorflow
print(tensorflow.__version__)

2.2.0-dlenv
